In [19]:
import numpy as np
import pandas as pd
from numpy.linalg import solve
import matplotlib.pyplot as plt
from scipy.linalg import qr
from scipy import optimize
import scipy as sp
from scipy.special import erfinv
from scipy.optimize import minimize_scalar
import egrss 
import lambda_fun

N = 100
sigma = 0.1
a, b = -0.2, 0.5
p = 2  # degree of splines
xi = (b-a) ** (2 * p - 1)
x = np.linspace(a,b,N)
Ut, Vt = egrss.generators((x-a)/(b-a),p)

def y(x):
    return 0.25*np.cos(4*np.pi *x) + 0.15*np.sin(12* np.pi *x) + 0.2 * x

np.random.seed(1)
yhat = y(x) + sigma * np.random.normal(loc = 0, scale = 1, size = x.shape)

# Finding the optimal lambda - GML
res_GML = lambda_fun.min_GCV(N, a, b, p, yhat, 'bounded', 'min',  (1,13))
lam = 1/np.power(10,res_GML.x) 
Wt, z  = egrss.potrf(Ut,Vt,N*lam/xi)
c,d    = lambda_fun.smoothing_spline_reg(Ut,Wt,z,yhat)

def phi(x, k, a):
    value = ((x-a)**(k-1))/np.math.factorial(k-1)
    return value

def extrapolate(x, yhat, a, b, c, d, p, n):
    value = 0
    for i in range(p):
            value += d[i]*phi(x, i+1, a)
    
    for i in range(n):
        value += d[i]*sp.integrate.quad((max(0,yhat[n]-u)**(p-1)/np.math.factorial(p-1))*
                                     (max(0,x-u)**(p-1)/np.math.factorial(p-1)), a, b)
    
    return value

value = extrapolate(0.6, yhat, a, b, c, d, p, N)

AttributeError: module 'scipy' has no attribute 'integrate'